In [ ]:
import torch
import numpy as np
import pandas as pd
from torch import cuda
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
import time
device = 'cuda' if cuda.is_available() else 'cpu'
print(device)

In [ ]:
ChunkSize = 2500000
i = 1
for chunk in pd.read_csv("unsw_1_labelled_flows.csv", chunksize=ChunkSize):
    #chunk.dropna(subset=['ip.src', 'ip.dst'],inplace=True)
    #chunk.drop(['tcp.seq','tcp.ack','ip.ttl','ip.hdr_len'], axis=1, inplace=True)
    dtypes = {'_ws.col.Time': 'float64',
          'ip.src':'str',
          'ip.dst':'str',
          'ip.hdr_len':'uint32',
          'ip.dsfield': 'uint32',
          'ip.flags': 'uint32',
          'ip.len': 'uint32',
          'ip.ttl': 'uint32',
          'ip.proto': 'uint32',
          'packet.len': 'uint32',
          'tcp.dstport': 'uint32',
          'tcp.srcport': 'uint32',
          'tcp.flags': 'uint32',
          'tcp.seq': 'uint32',
          'tcp.ack': 'uint32',
          'tcp.window_size_value': 'uint32',
          'tcp.payload':'object',
          'tcp.label':'uint32',
          'flow_id':'uint64',
          'subflow': 'uint32',
          'direction': 'int32',
          'payload_0' : 'uint32',
          'payload_1' : 'uint32',
          'payload_2' : 'uint32',
          'payload_3' : 'uint32',
          'payload_4' : 'uint32',
          'payload_5' : 'uint32',
          'Attack category': 'str',
          }

    for col in chunk.columns:
        chunk[col] = chunk[col].astype(dtypes[col])

    if i == 1:
        data = chunk
    else:
        data = pd.concat([data, chunk])
    print('-->Read Chunk...', i)
    i += 1



In [ ]:
data.loc[data['Attack category'] != 'Benign', 'Attack category'] = 1
data.loc[data['Attack category'] == 'Benign', 'Attack category'] = 0
data.rename(columns={'Attack category' : "label"}, inplace=True)
data = data[data['ip.proto'].isin([6,17,1,2])]

In [ ]:
data.sort_values(by=['_ws.col.Time'],inplace=True, ignore_index = True)
data = data.groupby('flow_id').head(512)
data.drop_duplicates(data.columns.delete(0).delete(6).delete(13),inplace = True)

In [ ]:
data = data.groupby('flow_id').head(32)


def get_rel_time():
    data['_ws.col.Time'] = data.groupby('flow_id')['_ws.col.Time'].transform(lambda x:  x - x.iloc[0])

get_rel_time()

data.drop(data[data['_ws.col.Time'] > 120].index,inplace=True)


data['label'] = data.groupby('flow_id')['label'].transform('max')
data.drop('tcp.ack', axis=1,inplace=True)
data.drop('tcp.seq', axis=1,inplace=True)
data.drop('ip.ttl', axis=1,inplace=True)

def get_rel_time():
    data['_ws.col.Time'] = data.groupby('flow_id')['_ws.col.Time'].diff().fillna(0)

get_rel_time()

In [ ]:
drop_columns = {"ip.src", "ip.dst",'udp.srcport', 'tcp.srcport','Unnamed: 0', 'tcp.dstport'}
cat_fields = ['ip.proto', 'ip.flags','tcp.flags','ip.dsfield']
mixed_cols = {'tcp.flags','ip.flags','ip.dsfield'}
payload_fields = ['payload_' + str(i) for i in range(6)]
data.reset_index(drop=True,inplace=True)
#for col in mixed_cols:
#    data[col] = data[col].map(lambda hexStr: int(hexStr, 16), na_action="ignore")

#data.dropna(axis=0, how='any', inplace=True)
#data.drop_duplicates(subset=None, keep="first", inplace=True)

label_field = {'label'}
num_fields = set(data.columns) - set(cat_fields) - label_field - {"iat"} -  {"_ws.col.Time"} - {"flow_id"} - {"direction"} - {"subflow"} - drop_columns - set(payload_fields) - {'group_size'} - {'group_size_pkt'}
num_fields = sorted(num_fields)
labels = data['label'].nunique()
print(data['label'].value_counts())

In [ ]:
from sklearn.preprocessing import KBinsDiscretizer
from sklearn.preprocessing import MaxAbsScaler
from sklearn.pipeline import Pipeline
DATE_FORMAT_DATASET = '%Y-%m-%d %H:%M:%S.%f'

np.random.seed(52)
torch.manual_seed(52)
torch.cuda.manual_seed(52)


# Select the length of the packet sequence
seq_len = 1024
unique_flows = data['flow_id'].unique()

def get_train_test_mask(data, seq_len):
    train_mask = np.full(data.shape[0], False)
    test_mask = np.full(data.shape[0], False)
    train_sample = set()
    test_sample = set()
    for i in data['label'].unique():
        group = data[data['label'] == i]['flow_id'].unique()
        eval_sample = np.random.choice(group, len(group), replace=False)
        if i == 0:
            train_sample |= (set(eval_sample[:round(0.6*len(eval_sample))]))
            test_sample |= (set(eval_sample[round(0.6*len(eval_sample)):]))
        elif i == 1:
            test_sample |= (set(eval_sample))
    train_mask[data.index[data['flow_id'].isin(train_sample)].to_list()] = True
    test_mask[data.index[data['flow_id'].isin(test_sample)].to_list()] = True
    return train_mask,test_mask,train_mask ,np.array(list(train_sample)),np.array(list(test_sample))


def fit_numerical(new_data, num_fields, eval_mask):
    for col in num_fields:
        print(col)
        col_values = data[col].values
        col_values[~np.isfinite(col_values)] = 0
        col_values = col_values.astype("uint32")
        new_data[col] = col_values

def fit_categorical(new_data, cat_fields, eval_mask):
    for col in cat_fields:
        print(col)
        pre_values = data[col].values
        new_data[col] =  pre_values + 3

        print(col,np.unique(new_data[col]))

def create_new_df(num_fields, cat_fields, eval_mask, seq_len):
    new_data = {}
    new_data['flow_id'] = data['flow_id']
    new_data['_ws.col.Time'] = data['_ws.col.Time']
    new_data['direction'] = data['direction'].astype("int32") + 3

    print(new_data['_ws.col.Time'])
    fit_categorical(new_data, cat_fields, eval_mask)
    fit_numerical(new_data, num_fields, eval_mask)
    for f in payload_fields:
        new_data[f] = data[f].apply(lambda x: x + 1 if x > 1 else x).astype("int32") 
    return pd.DataFrame(new_data)

train_mask,test_mask,fit_mask,train_idx,test_idx = get_train_test_mask(data, seq_len)
new_df = create_new_df(num_fields, cat_fields, fit_mask, seq_len)

In [ ]:
batch_size = 128

def generate_batches(train_mask, test_mask, batch_size):
    batches_train = np.random.choice(train_mask, (len(train_mask)//batch_size, batch_size), replace=False)
    batches_test = np.random.choice(test_mask, (len(test_mask)//128, 128), replace=False)
    return batches_train, batches_test

batches_train,batches_test = generate_batches(train_idx,test_idx,batch_size)
new_df = new_df.join(data['label'])
new_df['count'] = new_df['flow_id'].map(new_df['flow_id'].value_counts())

In [ ]:
new_df.drop('payload_0', axis=1,inplace=True)
new_df.drop('payload_1', axis=1,inplace=True)
new_df.drop('payload_2', axis=1,inplace=True)
new_df.drop('payload_3', axis=1,inplace=True)
new_df.drop('payload_4', axis=1,inplace=True)
new_df.drop('payload_5', axis=1,inplace=True)
new_df['_ws.col.Time'] = new_df['_ws.col.Time'] 
new_df['packet.len'] = new_df['packet.len'] / 1400
new_df['_ws.col.Time'] = new_df['_ws.col.Time'] * 10 ** 3

In [ ]:
from sklearn.preprocessing import MaxAbsScaler,MinMaxScaler,StandardScaler
new_df.index.name = 'index'
new_df.sort_values(by=['flow_id', 'index'],inplace=True)
new_df.set_index('flow_id', inplace=True)
new_df.drop('ip.len', axis=1,inplace=True)
new_df.drop('ip.dsfield', axis=1,inplace=True)
new_df.drop('tcp.window_size_value', axis=1,inplace=True)
new_df.drop('ip.flags', axis=1,inplace=True)
new_df.drop('ip.hdr_len', axis=1,inplace=True)


In [ ]:
## SLICING
train_df = new_df[new_df.index.isin(train_idx)]
test_df = new_df[new_df.index.isin(test_idx)]

counts_indexes = {}
for i in range(1,33):
    counts_indexes[i] = train_df[train_df['count'] == i].index

train_df.drop('count', axis=1,inplace=True)
test_df.drop('count', axis=1,inplace=True)
new_df.drop('count', axis=1,inplace=True)

In [ ]:
torch.set_num_threads(1)
labels = 1
## SLICING
def batch_loader(batches, device, new_df, mask = True):
    matching_rows_list = []
    label_rows_list = []
    matching_rows_list_pos = []
    mask_rows = []
    mask_rows_2 = []
    sliced_df = new_df[new_df.index.isin(batches)]
    unique_combinations = sliced_df.groupby('flow_id')

    # Iterate over the groups
    for group_name, group in unique_combinations:
        seq_x = group.to_numpy()[:32,:-labels]
        if mask:
            seq_x_1 = seq_x.copy()

            

            matching_rows_list.append(torch.tensor(seq_x_1.astype('float32'), device=device))
            mask_rows.append(torch.ones(seq_x_1.shape[0] + 1, device=device))

            seq_x_2 = seq_x.copy()
            if seq_x.shape[0] > 1:
                new_seq = new_df.loc[np.random.choice(counts_indexes[seq_x.shape[0]])].to_numpy()
                if new_seq.ndim == 1:
                    new_seq = new_seq.reshape(1, -1)
                new_seq = new_seq[:32,:-labels]

                # Choose a random cut length (avoiding full replacement)
                cut_length = round(0.4 * (seq_x.shape[0]))
                # Choose a random starting position in both embeddings
                start_a = np.random.randint(0, seq_x.shape[0] - cut_length + 1)

                
                
                # Create the new mixed embedding by replacing a segment from emb_a with emb_b
                seq_x_2[start_a:start_a + cut_length] = new_seq[start_a:start_a + cut_length]
            
            
            matching_rows_list_pos.append(torch.tensor(seq_x_2.astype('float32'), device=device))
            mask_rows_2.append(torch.ones(seq_x_2.shape[0] + 1, device=device))
        else:

            matching_rows_list.append(torch.tensor(seq_x.astype('float32'), device=device))
            label_rows_list.append(torch.tensor(group.to_numpy()[0,-labels:].astype('float32'), device=device))
            mask_rows.append(torch.ones(seq_x.shape[0] + 1, device=device))



    tensor_x = torch.nn.utils.rnn.pad_sequence(matching_rows_list, batch_first=True, padding_value=0)
    cls_token =  torch.cat((torch.zeros(tensor_x.shape[0], 1, 1, device=device, dtype=torch.float), torch.ones(tensor_x.shape[0], 1, 4, device=device, dtype=torch.float)), 2)
    tensor_x = torch.cat((cls_token,tensor_x), 1)
    tensor_x[:,1,0] =  0
    tensor_x[:,0,4] =  0

    if mask:
        tensor_y = torch.nn.utils.rnn.pad_sequence(matching_rows_list_pos, batch_first=True, padding_value=0)
        cls_token =  torch.cat((torch.zeros(tensor_y.shape[0], 1, 1, device=device, dtype=torch.float), torch.ones(tensor_y.shape[0], 1, 4, device=device, dtype=torch.float)), 2)
        tensor_y = torch.cat((cls_token,tensor_y), 1)
        tensor_y[:,1,0] =  0
        tensor_y[:,0,4] =  0

        masked = torch.nn.utils.rnn.pad_sequence(mask_rows, batch_first=True, padding_value=0)
        masked_2 = torch.nn.utils.rnn.pad_sequence(mask_rows_2, batch_first=True, padding_value=0)
    else:
        tensor_y = torch.nn.utils.rnn.pad_sequence(label_rows_list, batch_first=True, padding_value=0)
        masked = torch.nn.utils.rnn.pad_sequence(mask_rows, batch_first=True, padding_value=0)
        masked_2 = masked
    
    masked = ~masked.to(torch.bool)
    masked_2 = ~masked_2.to(torch.bool)
    #print(count)

    return tensor_x,tensor_y,masked,masked_2

#diff_avg_all = []
for i, batch in enumerate(batches_train):
    Xbatch,ybatch,mask,mask_2 = batch_loader(batch, device, train_df)
    print(Xbatch.shape)
    print(Xbatch[0,:,0])
    print(ybatch[0,:,0])
    if i == 1:
        break



In [ ]:
hidden_dim = 256

class EmbeddingLayer(torch.nn.Module):
    def __init__(self):
      super(EmbeddingLayer, self).__init__()
      self.position_embeddings = torch.nn.Embedding(34, hidden_dim)
      self.embedding_layer = torch.nn.ModuleList([torch.nn.Embedding(65539, 51) for col in new_df.columns[1:-(labels + 1)]])
      self.time_layer = torch.nn.Linear(1,51,bias=False)
      self.time_layer_2 = torch.nn.Linear(128,51)

      self.packet_layer = torch.nn.Linear(1,51,bias=False)
      self.packet_layer_2 = torch.nn.Linear(128,51)

      self.proj_layer = torch.nn.Linear(51 * (new_df.shape[-1]  - labels), hidden_dim)
      torch.nn.init.xavier_uniform_(self.proj_layer.weight)
      self.proj_layer.bias.data.fill_(0)
      self.activ = torch.nn.ReLU()
      self.norm_3 = torch.nn.LayerNorm(256)
      self.norm_2 = torch.nn.LayerNorm(45 * (new_df.shape[-1]  - labels - 1))
      self.norm_1 = torch.nn.LayerNorm(273,eps=1e-12)
      self.dropout = torch.nn.Dropout(0.1)
    
    
    def forward(self, input):
        position = torch.arange(input.shape[1], dtype=torch.long, device=device)
        position = position.unsqueeze(0).expand((input.shape[0],input.shape[1]))


        list_emb = self.time_layer(input[:, :, [0]])
        packet_emb = self.packet_layer(input[:, :, [4]])
        list_emb_2 = [self.embedding_layer[i-1](input[:, :, i].int()) for i in range(1,input.shape[-1] - 1)]

        embed_tokens = self.proj_layer(torch.cat((list_emb,packet_emb,torch.cat(list_emb_2,dim=2)),dim=2))
        hidden_states = embed_tokens + self.position_embeddings(position)
        return hidden_states
  

class OutputLayer(torch.nn.Module):
  def __init__(self):
    super(OutputLayer, self).__init__()
    self.activ = torch.nn.ReLU()
    self.linear =  torch.nn.Linear(hidden_dim,hidden_dim)
    self.linear_2 =  torch.nn.Linear(hidden_dim,hidden_dim)
  
  def forward(self, input):
      cls = self.linear_2(self.activ(self.linear(input)))
      return cls

class CLSLayer(torch.nn.Module):
  def __init__(self):
    super(CLSLayer, self).__init__()
    self.activ = torch.nn.ReLU()
    self.sig_activ = torch.nn.Sigmoid()
    self.linear =  torch.nn.Linear(hidden_dim,4*hidden_dim)
    self.linear_2 =  torch.nn.Linear(4*hidden_dim,hidden_dim)
    self.linear_3 =  torch.nn.Linear(hidden_dim,1)
  
  def forward(self, input):
      cls = self.sig_activ(self.linear_3(self.linear_2(self.activ(self.linear(input)))))
      return cls
  

class BERT(torch.nn.Module):
  def __init__(self):
    super(BERT, self).__init__()
    self.norm = torch.nn.LayerNorm(768)
    self.embed = EmbeddingLayer()
    self.encoder_layer = torch.nn.TransformerEncoderLayer(hidden_dim, 4, 256*4, batch_first=True, activation = 'gelu')
    self.encoder = torch.nn.TransformerEncoder(self.encoder_layer, num_layers=4)
    self.out_layer = OutputLayer()
    self.cls_layer = CLSLayer()
  
  def forward(self, input_1, input_2, mask):
      embed_1 = self.embed(input_1)
      enc = self.encoder(embed_1,src_key_padding_mask=mask)
      cls_1 = self.out_layer(enc[:, 0, :])
      
      embed_2 = self.embed(input_2) 
      enc = self.encoder(embed_2,src_key_padding_mask=mask)
      cls_2 = self.out_layer(enc[:, 0, :])
      return cls_1,cls_2
  
  def embeddings(self, input, mask):
      embed = self.embed(input)
      enc = self.encoder(embed,src_key_padding_mask=mask)
      return enc[:, 0, :]

  def embeddings_cls(self, input, mask):
      embed = self.embed(input)
      enc = self.encoder(embed,src_key_padding_mask=mask)
      return self.cls_layer(enc[:, 0, :])


class CosineWarmupScheduler(torch.optim.lr_scheduler._LRScheduler):

    def __init__(self, optimizer, warmup, max_iters):
        self.warmup = warmup
        self.max_num_iters = max_iters
        super().__init__(optimizer)

    def get_lr(self):
        lr_factor = self.get_lr_factor(epoch=self.last_epoch)
        return [base_lr * lr_factor for base_lr in self.base_lrs]

    def get_lr_factor(self, epoch):
        lr_factor = 0.5 * (1 + np.cos(np.pi * epoch / self.max_num_iters))
        if epoch <= self.warmup:
            lr_factor *= epoch * 1.0 / self.warmup
        return lr_factor
        
model = BERT()
model = model.to(device)
optimizer = torch.optim.AdamW(model.parameters(), 0.00005)

In [ ]:
import torch.nn.functional as F

class NTXent(torch.nn.Module):
    def __init__(self, temperature=0.5):
        """NT-Xent loss for contrastive learning using cosine distance as similarity metric as used in [SimCLR](https://arxiv.org/abs/2002.05709).
        Implementation adapted from https://theaisummer.com/simclr/#simclr-loss-implementation

        Args:
            temperature (float, optional): scaling factor of the similarity metric. Defaults to 1.0.
        """
        super().__init__()
        self.temperature = temperature

    def forward(self, z_i, z_j):
        """Compute NT-Xent loss using only anchor and positive batches of samples. Negative samples are the 2*(N-1) samples in the batch

        Args:
            z_i (torch.tensor): anchor batch of samples
            z_j (torch.tensor): positive batch of samples

        Returns:
            float: loss
        """
        batch_size = z_i.size(0)
        #z_i = F.normalize(z_i, p=2, dim=1)
        #z_j = F.normalize(z_j, p=2, dim=1)

        # compute similarity between the sample's embedding and its corrupted view
        z = torch.cat([z_i, z_j], dim=0)
        similarity = F.cosine_similarity(z.unsqueeze(1), z.unsqueeze(0), dim=2)

        sim_ij = torch.diag(similarity, batch_size)
        sim_ji = torch.diag(similarity, -batch_size)
        positives = torch.cat([sim_ij, sim_ji], dim=0)  # size is 2*bs

        mask = (
            ~torch.eye(batch_size * 2, batch_size * 2, dtype=torch.bool)
        ).float().to(device)
        numerator = torch.exp(positives / self.temperature)
        denominator = mask * torch.exp(similarity / self.temperature)

        all_losses = -torch.log(numerator / torch.sum(denominator, dim=1))
        loss = torch.sum(all_losses) / (2 * batch_size)

In [ ]:
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve, auc

def model_accuracy(model, device, y_pred = None):
    accuracy = 0.0
    running_vloss = 0.0
    inf_time = 0.0
    y_label = []
    y_test = None
    cos_sim_out = None
    
    with torch.no_grad():
        if y_pred == None:
            for i, batch in enumerate(batches_train):
                tensor_x_test, tensor_y_test, mask, mask_2 = batch_loader(batch, device, train_df, mask= False)
                
                # Measure inference time for GPU
                if device == 'cuda':
                    starter, ender = torch.cuda.Event(enable_timing=True), torch.cuda.Event(enable_timing=True)
                    starter.record()
                    y_out = model.embeddings(tensor_x_test,mask).clone()
                    ender.record()
                    torch.cuda.synchronize()
                    inf_time += starter.elapsed_time(ender)
                else: # Or for CPU
                    start = time.time()
                    y_out = model(tensor_x_test)
                    end = time.time() - start
                    inf_time += end * 1000
                    
                if y_pred == None:
                    y_pred = y_out
                else:
                    y_pred = torch.cat([y_pred,y_out])


            y_pred = F.normalize(y_pred, p=2, dim=1)

        cos_sim_lst = []
        roc_score = 0

        for i, batch in enumerate(batches_test):
            tensor_x_test, tensor_y_test, mask, mask_2 = batch_loader(batch, device, test_df, mask= False)
            
            # Measure inference time for GPU
            if device == 'cuda':
                starter, ender = torch.cuda.Event(enable_timing=True), torch.cuda.Event(enable_timing=True)
                starter.record()
                y_out = F.normalize(model.embeddings(tensor_x_test,mask).clone(), p=2, dim=1)
                similarity = torch.mm(y_out, y_pred.t())
                cos_sim,indices = similarity.max(dim=1)
                ender.record()
                torch.cuda.synchronize()
                inf_time += starter.elapsed_time(ender)
                cos_sim_lst.append(cos_sim.cpu())
            else: # Or for CPU
                start = time.time()
                y_out = model(tensor_x_test)
                end = time.time() - start
                inf_time += end * 1000
            
            if i == 0:
                for j in range(cos_sim.shape[0]):
                    print(j,indices[j],float(cos_sim[j]),int(1 - tensor_y_test[j]))
            
            if y_test == None:
                y_test = tensor_y_test.cpu()
                cos_sim_out = cos_sim.cpu()
            else:
                y_test = torch.cat([y_test,tensor_y_test.cpu()])
                cos_sim_out = torch.cat([cos_sim_out,cos_sim.cpu()])
            

        roc_score = roc_auc_score(y_test, 1 - cos_sim_out)
        fpr, tpr, thresholds = roc_curve(y_test, 1 - cos_sim_out)
        # Find the best threshold
        # Calculate Youden's J statistic
        youden_j = tpr - fpr
        best_threshold_index = np.argmax(youden_j)
        best_threshold = thresholds[best_threshold_index]
        print(accuracy_score(y_test, (1 - cos_sim_out) >= best_threshold))
        print(classification_report(y_test, (1 - cos_sim_out) >= best_threshold,digits=4))

        print("Best Threshold:", best_threshold)
        print("TPR:", tpr[best_threshold_index], "FPR:", fpr[best_threshold_index])


        

    print("ROC_AUC_SCORE",roc_score)
    return roc_score,y_pred

# Train the model
def train_model(model, epochs, lr, batches_train, device):
    loss_fn = NTXent()
    train_time = 0.0
    batch_arr = batches_train.copy()
    for epoch in range(epochs):
        running_loss = 0.
        batch_loss = 0.
        model.train(True)
        for i, batch in enumerate(batch_arr):
            if device == 'cuda':
                starter, ender = torch.cuda.Event(enable_timing=True), torch.cuda.Event(enable_timing=True)
                starter.record()
                loss = train_batch(model, loss_fn, optimizer, batch)
                ender.record()
                torch.cuda.synchronize()
                train_time += starter.elapsed_time(ender)
            else:
                start = time.time()
                loss = train_batch(model, loss_fn, optimizer, Xbatch, ybatch)
                end = time.time() - start
                train_time += end * 1000    

            running_loss += loss.item()
            batch_loss += loss.item()
            if i % 10 == 0:
                print("batch ", i, batches_train.shape, loss.item(), batch_loss / 10)
                batch_loss = 0.
            

        last_loss = running_loss / (i+1)
        print('epoch {} batches {} loss: {} time: {} ms'.format(epoch, i + 1, last_loss, train_time))
        batch_arr = batch_arr.reshape(-1,1)
        np.random.shuffle(batch_arr)
        batch_arr = batch_arr.reshape(batches_train.shape[0],batches_train.shape[1])
        model.eval()
        roc_score,y_pred = model_accuracy(model, device='cuda')
    return roc_score,y_pred
        
def train_batch(model, loss_fn, optimizer, batch):
    optimizer.zero_grad()
    Xbatch, xbatch_pair ,att_mask ,att_mask_2 = batch_loader(batch, device, train_df, mask= True)
    anchor,pair = model(Xbatch, xbatch_pair,att_mask)
    loss = loss_fn(anchor, pair)
    loss.backward()
    optimizer.step()
    #scheduler.step()  # Update the learning rate
    return loss

roc_score,y_pred = train_model(model, epochs=1, lr=0.0001, batches_train=batches_train, device='cuda')